In [3]:
# Imports
import pandas as pd
import gensim
import numpy as np
import nltk
import re

from helpers  import *

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import remove_stopwords
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *

#nltk.download('punkt')

np.random.seed(2018) # set random seed


In [7]:
# Load data
tweets1 = pd.read_csv(access_folder('data') + 'IRAhandle_tweets_1.csv')

# selecting content columns for subject categorization (all should be in english)
content = tweets1[tweets1.language == 'English'].content

content

0         "We have a sitting Democrat US Senator on tria...
1         Marshawn Lynch arrives to game in anti-Trump s...
2         Daughter of fallen Navy Sailor delivers powerf...
3         JUST IN: President Trump dedicates Presidents ...
4         19,000 RESPECTING our National Anthem! #StandF...
5         Dan Bongino: "Nobody trolls liberals better th...
6                               🐝🐝🐝 https://t.co/MorL3AQW0z
7         '@SenatorMenendez @CarmenYulinCruz Doesn't mat...
8         As much as I hate promoting CNN article, here ...
9         After the 'genocide' remark from San Juan Mayo...
10        After the 'genocide' remark from San Juan Mayo...
11        '@thehill Why won't she apologize to us for ly...
12        Sarah Sanders destroys NBC reporter: "Trump ma...
13        Hi @MichelleObama, remember when you said Wein...
14        Hi @MichelleObama, remember when you praised H...
15        Wow! Even CNN is slamming the Obamas for silen...
16        First lady Melania Trump visit

In [ ]:
# Tokenization: Split text into words. Lowercase the words and remove punctuation

stemmer = SnowballStemmer("english") # initiate dictionary type
# create words from sentences
content_tokens = [nltk.word_tokenize(x) for x in content]
# Stem words (truncate)
content_stemmed = []
for tweet in content_tokens:
    stem = [stemmer.stem(x) for x in tweet if len(x) > 3] # only consider words more than 3 letters
    content_stemmed.append(stem)

In [ ]:
# Create dictionary to know the frequency of words
dictionary = gensim.corpora.Dictionary(content_stemmed)
dictionary.filter_extremes(no_below=15) # remove words that have appearances less than 15 times

# Create bag-of-words
bow_corpus = [dictionary.doc2bow(doc) for doc in content_stemmed]

In [ ]:
# Check shit is working
test = bow_corpus[126]
for i in range(len(test)):
    print("Word {} (\"{}\") appears {} time.".format(test[i][0],dictionary[test[i][0]], 
test[i][1]))

In [ ]:
# Create model on bag-of-words

lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [ ]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))